In [1]:
import numpy as np
import math
import glob
import pandas as pd
import folium
import json
from itertools import combinations 

# These lines set up the plotting functionality and formatting.
# import matplotlib
# matplotlib.use('Agg', warn=False)
# %matplotlib inline
# import matplotlib.pyplot as plots
# plots.style.use('fivethirtyeight')
# import warnings
# warnings.simplefilter(action="ignore", category=FutureWarning)


# 1. Preparing the Data

The repository https://opendata.charlottesville.org/ offers data on various utilities in the City of Charlottesville. The Property subsection itself has various datasets. Thus, it is important to first obtain only portions of each table that are relevant.

## 1.1 Reading/Modifying Sales Table

The Sales Table contains data about sales of particular parcels. It contains their addresses, date of sale, and amount of sale. Firstly, the Street Number and Street Address are combined for ease of use later on. 

In [2]:
sales = pd.read_csv('Data/Real_Estate__Sales_.csv')

sales['Combo'] = sales['StreetNumber'] + ' '+sales['StreetName'] 

A function is written to modify the SaleDate column of the dataframe. The time portion is removed since none of the dates appear to specify an actual time. The backslashes are replaced with dashes to better coincide with numpy's date objects.

In [3]:
def formatDate(row):  
    date = row['SaleDate']
    return date[:10].replace('/','-')

sales = sales[~pd.isna(sales.SaleDate)]
sales = sales.assign(SaleDate= sales.apply(formatDate,axis=1))

## 1.2 Merging Sales with Residential

The Residential datasheet contains a list of all the residential parcels of the area. This is important to filter out the non-residential parcels located in the Sales dataset. Merge the Sales table with the Residential datasheet. The join is done using the ParcelNumber from each dataframe.

In [4]:
# Filter out non-residential parcels
resid = pd.read_csv('Data/Real_Estate__Residential_Details_.csv',header=0)

salesResid = pd.merge(sales,resid.ParcelNumber,on="ParcelNumber")

## 1.3 Merging with Geocoded Addresses

The addresses in the table were then geocoded (found latitude/longitude coordinates using addresses) and located in a .csv file called 'coordinates.csv'. This was merged with the combined Residential Sales table. geoSalesResid is the table that will be used for subsequent sections as the proper changes have been made

In [5]:
geocoded = pd.read_csv('coordinates.csv')

geoSalesResid = pd.merge(salesResid,geocoded,left_on="Combo",right_on="ADDRESS")
geoSalesResid = geoSalesResid.drop(columns=["StreetName","StreetNumber","Combo"])
geoSalesResid

,RecordID_Int,ParcelNumber,SaleDate,SaleAmount,Unit,ADDRESS,LATITUDE,LONGITUDE
0,23,010001600,1900-01-01,0,NaN,600 MASSIE RD,38.054079,-78.507724
1,58,010005000,1993-02-18,0,NaN,0 BARRACKS RD,38.055077,-78.500401
2,59,010005000,1993-02-18,0,NaN,0 BARRACKS RD,38.055077,-78.500401
3,64,010008000,1984-08-31,175000,NaN,0 BARRACKS RD,38.055077,-78.500401
4,65,010008000,2015-01-28,0,NaN,0 BARRACKS RD,38.055077,-78.500401
...,...,...,...,...,...,...,...,...
52772,56277,610318000,2016-12-05,0,NaN,110 MILFORD TER,38.018268,-78.470688
52773,56278,610318000,1999-10-21,81500,NaN,110 MILFORD TER,38.018268,-78.470688
52774,56279,610318000,2017-03-07,148000,NaN,110 MILFORD TER,38.018268,-78.470688
52775,56280,610318000,2003-03-03,116900,NaN,110 MILFORD TER,38.018268,-78.470688


# 2. Finding Boundaries

The goal of this section is to recognize regions (neighborhoods) in Charlottesville that share similar housing prices. recentSales below represents non-zero sales that have occured after a certain date.

## 2.1 Introduction

In [6]:
cutoffDate = '2018-12-31'
recentSales = geoSalesResid[(geoSalesResid.SaleAmount>0) & (geoSalesResid.SaleDate > cutoffDate)]
recentSales

,RecordID_Int,ParcelNumber,SaleDate,SaleAmount,Unit,ADDRESS,LATITUDE,LONGITUDE
62,114,010022000,2019-09-04,1300000,NaN,1882 WESTVIEW RD,38.049146,-78.499856
114,163,010034000,2019-02-15,470000,NaN,1882 FIELD RD,38.049078,-78.501232
125,174,010038000,2019-03-29,1375000,NaN,1868 FIELD RD,38.048151,-78.500521
140,189,010041000,2019-06-06,1350000,NaN,906 FENDALL TER,38.047800,-78.501099
147,196,010043000,2020-01-07,680000,NaN,1861 WINSTON RD,38.047481,-78.500629
...,...,...,...,...,...,...,...,...
52642,56147,610291000,2019-10-31,200000,NaN,104 DANBURY CT,38.018508,-78.471445
52651,56156,610292000,2019-11-12,119000,NaN,106 DANBURY CT,38.018453,-78.471450
52682,56187,610299000,2020-02-04,188600,NaN,120 DANBURY CT,38.018009,-78.471492
52734,56239,610309000,2019-04-19,203500,NaN,107 DANBURY CT,38.018429,-78.470994


The chooseColor function defines the sale cutoffs that will be used on the map. The makeMarker will take each row of recentSales and produce a marker based on the SaleAmount

In [13]:
def chooseColor(sale):
    if sale < 89000:
        return 'purple'
    elif 89000<= sale < 150000:
        return 'blue'
    elif 150000<= sale < 400000:
        return 'green'
    elif 400000<= sale < 800000:
        return 'orange'
    else:
        return 'red'

def makeMarkerArea(row,currmap):
    line = row["SaleDate"][:4] + ": $" + str(row['SaleAmount'])
    folium.CircleMarker(location=[row['LATITUDE'],row['LONGITUDE']],
                  popup=line,radius=1.5,color=chooseColor(row['SaleAmount']),
                       fill_color=chooseColor(row['SaleAmount'])).add_to(currmap)
    return

A Folium map is created, centered on Charlottesville. Before regions can be created, the map should first be viewed with recent sales to determine any patterns. For speed, a map is created with the sales that have occured from 2019 onwards.

In [14]:
m1 = folium.Map(location=[38.0293, -78.4767], zoom_start=13)
recentSales.iloc[:].apply(makeMarkerArea,axis=1,args=(m1,))
m1

From the above map, it can be seen that there are exist regions that share SaleAmounts of similar magnitude. The webtool https://geojson.io/ offers a GUI-based method of producing JSON polygons for a map. The Folium package has a method of reading in these json files as a list of lists that represent the polygon by its vertices. The pandas read_json function gives a dataframe of this information, and is then removed of unnecessary information.

In [9]:
def extractCoord(row):
    return row['features']['geometry']['coordinates']
lay = pd.read_json('regions.json')
lay = lay.apply(extractCoord,axis=1)
lay

0     [[-78.49175691604614, 38.062081093980034], [-7...
1     [[-78.47821712493896, 38.06439561105793], [-78...
2     [[-78.48898887634277, 38.056750668544986], [-7...
3     [[-78.46072375774384, 38.04977660181616], [-78...
4     [[-78.45959186553955, 38.034977214269404], [-7...
5     [[-78.47529888153075, 38.042446990693016], [-7...
6     [[-78.47525596618652, 38.0295518538066], [-78....
7     [[-78.48570585250854, 38.023348535033705], [-7...
8     [[-78.49637031555176, 38.02454867310244], [-78...
9     [[-78.48905324935913, 38.02578259738154], [-78...
10    [[-78.51654052734375, 38.0286306857249], [-78....
11    [[-78.50675582885742, 38.03982758570638], [-78...
12    [[-78.50548982620239, 38.04339333332387], [-78...
dtype: object

## 2.2 Point in Polygon Algorithm

With the imported Json polygon layers, each point in the previous section should be grouped with each polygon. This is a point in polygon problem: determine whether or not a point is in a polygon defined by its vertices. 

A way to solve this problem is with ray-casting. Take a point and extend it "infinitely" along an arbitrary direction (it is now a ray). Count the number of times the point intersects a side of the polygon. If the number of ray intersections is even, then it is not in the polygon. If the number of ray intersections is odd, then it is in the polygon. 

![Solving PIP with Ray-casting](https://upload.wikimedia.org/wikipedia/commons/c/c9/RecursiveEvenPolygon.svg)


The first issue is, given two pairs of points (that define two line segments), determine if they intersect or not. The solution is to check the orientation of these points (intersecting line segments should have their respective points in between one point from the other line segment). A algorithm written by https://kite.com/python/answers/how-to-check-if-two-line-segments-intersect-in-python is used. The main function below is intersects(s1,s2) and it takes in two tuples.

In [10]:
def on_segment(p,q,r):
    if r[0] <= max(p[0], q[0]) and r[0] >= min(p[0], q[0]) and r[1] <= max(p[1], q[1]) and r[1] >= min(p[1], q[1]):
        return True
    else: 
        return False

def orientation(p,q,r):
    val = ((q[1] - p[1]) * (r[0] - q[0])) - ((q[0] - p[0]) * (r[1] - q[1]))
    if val == 0:
        return 0
    elif val > 0:
        return 1
    else:
        return -1

def intersects(s1,s2):
    p1,q1 =s1
    p2,q2 = s2
    
    o1 = orientation(p1,q1,p2)
    o2 = orientation(p1,q1,q2)
    o3 = orientation(p2,q2,p1)    
    o4 = orientation(p2,q2,q1)
    
    if o1 != o2 and o3 != o4:
        return True
    if o1 == 0 and on_segment(p1,q1,p2):
        return True
    if o2 == 0 and on_segment(p1, q1, q2):
        return True
    if o3 == 0 and on_segment(p2, q2, p1):
        return True
    if o4 == 0 and on_segment(p2, q2, q1):
        return True
    return False


Now the point in polygon algorithm can be applied for this scenario. The main function is pointinpolygons, which will be applied across the entire table with each row (point) as its argument. It will return a series of booleans showing whether or not the point is in each of the polygons in the Json file.

The makeray function receives a point and returns a sequence with two tuples inside that represent the ray. For this, each point is extended horizontally. Additionally, since all points reside in Charlottesville, the left/right longitude are the west-most/east-most points of the area, respectively. 

The pointinpolygon function is the individual function that is applied inside pointinpolygons. It creates a list of tuples that contain the line segments of the polygon. It is assumed that the Json file is configured so that the points can be connected from smallest to biggest (index 0 to index 1, index 1 to index 2, etc) until the very last which is connected back to index 0. It then counts the number of intersections as described above about the ray-casting. 

In [11]:
# defines a point, returns a series of whether or not point is in each JSON polygon
def pointinpolygons(row,layout):
    pt = (row['LATITUDE'],row['LONGITUDE'])
    rayseg = makeray(pt)
    
    foundin = layout.apply(pointinpoly,args=(rayseg,))
    if foundin[foundin].shape[0] == 0:
        return -1
    else:
        return foundin[foundin].index[0]
    

# produces a ray by extending a point horizontally depending on its longitude relative to Cville
def makeray(point):
    # Bounds for Longitude 
    leftb = -78.647930
    rightb = -78.411250
    midb = leftb + (rightb-leftb)/2

    if point[1] > midb:
        return (point,(point[0],leftb)) 
    else:
        return (point,(point[0],rightb))
    
# determines if the point is in or not in a single JSON polygon
def pointinpoly(polygons,raysegment):
    # Makes pairs of indices to form polygon edges (assumes that edges are defined as) from a polygon that has n sides
    # 0 to 1, 1 to 2, 2 to 3, etc... until the last index which would be n-1 to 0
    combo = []
    for i in np.arange(len(polygons)):
        if i < len(polygons) - 1:
            apair = (i,i+1)
        else:
            apair = (i,0)
        combo.append(apair)
    
    # Iterates through the list of pairs and indexes into the polygons variable which is a list of lists. It produces a 
    # a segment represented by two points and it is determined whether it intersects with the address.
    intersections = 0 
    for pair in combo:
        segment1 = ((polygons[pair[0]][1],polygons[pair[0]][0]), (polygons[pair[1]][1],polygons[pair[1]][0]))
        if intersects(segment1,raysegment):
            intersections +=1
            
    if intersections % 2 == 0:
        return False
    else:
        return True



The function is used to insert a column that assigns each point to its polygon region. If it is not in any of the polygons, then it is assigned -1. 

In [12]:
recentSales.insert(recentSales.shape[1],'Region',recentSales.apply(pointinpolygons,axis=1,args=(lay,)))
recentSales


,RecordID_Int,ParcelNumber,SaleDate,SaleAmount,Unit,ADDRESS,LATITUDE,LONGITUDE,Region
62,114,010022000,2019-09-04,1300000,NaN,1882 WESTVIEW RD,38.049146,-78.499856,12
114,163,010034000,2019-02-15,470000,NaN,1882 FIELD RD,38.049078,-78.501232,12
125,174,010038000,2019-03-29,1375000,NaN,1868 FIELD RD,38.048151,-78.500521,12
140,189,010041000,2019-06-06,1350000,NaN,906 FENDALL TER,38.047800,-78.501099,12
147,196,010043000,2020-01-07,680000,NaN,1861 WINSTON RD,38.047481,-78.500629,12
...,...,...,...,...,...,...,...,...,...
52642,56147,610291000,2019-10-31,200000,NaN,104 DANBURY CT,38.018508,-78.471445,7
52651,56156,610292000,2019-11-12,119000,NaN,106 DANBURY CT,38.018453,-78.471450,7
52682,56187,610299000,2020-02-04,188600,NaN,120 DANBURY CT,38.018009,-78.471492,7
52734,56239,610309000,2019-04-19,203500,NaN,107 DANBURY CT,38.018429,-78.470994,7


To verify the accuracy of the algorithm, the same table is used to create another map with the Json layers. Filtering out those without a region, there does not seem to be a point that lies outside the polygons. 

In [16]:
m2 = folium.Map(location=[38.0293, -78.4767], zoom_start=13)
testTab = recentSales[recentSales.Region>=0].apply(makeMarkerArea,axis=1,args=(m2,))
folium.GeoJson("regions.json", name='area').add_to(m2)
m2

## 2.3 Grouping the points

The pandas group function can be used to group all the points by region and determine the average sale amount within that region.

In [21]:
recentSales.groupby(['Region']).mean().loc[:,"SaleAmount"][1:]

Region
0     2.729115e+05
1     3.845725e+05
2     4.865075e+05
3     3.970500e+05
4     4.727176e+05
5     6.826686e+05
6     3.219663e+05
7     3.131621e+05
8     1.203167e+05
9     6.098358e+05
10    4.702161e+05
11    7.340401e+05
12    1.052130e+06
Name: SaleAmount, dtype: float64

# 3. Sales History Aggregation

A couple of functions are written to provide different ways of grouping the table above for later use. 
- "normal": used to clump up all listed sales under a possible parcel as a dictionary. 
- "byYear": adds a condition of only including sales after a specified year (set as a default parameter). 

In [ ]:
def normal(parcel):
    temp = {}

    if parcel.shape[0] == 1:
        saledate = parcel.iloc[0,2]
        temp[saledate] = parcel.iloc[0,3]
    else:
        for i in np.arange(parcel.shape[0]):
            saledate = parcel.iloc[i,2]
            temp[saledate] = parcel.iloc[i,3]
    return temp

def byYear(parcel,year=2000):
    temp = {}

    if parcel.shape[0] == 1:
        saledate = parcel.iloc[0,2]
        if int(saledate[:4]) >= year:
            temp[saledate] = parcel.iloc[0,3]
    else:
        for i in np.arange(parcel.shape[0]):
            saledate = parcel.iloc[i,2]           
            if int(saledate[:4]) >= year:
                temp[saledate] = parcel.iloc[i,3]
    return temp



The "getSalesHistory" function will return a dictionary of the sales in the table depending on the desired form of filtering (see above) list of functions.

In [ ]:
def getSalesHistory(table, norm=True, multi=False):
    grouped = geoSalesResidFilterSale.groupby('ParcelNumber')
    if norm:
        history = grouped.apply(normal).to_dict()
    else:
        temp = grouped.apply(byYear).to_dict()
        history = {}
        for parcel in temp:
            if len(temp[parcel]) > 1:
                history[parcel] = temp[parcel]
            elif not multi and len(temp[parcel])==1:
                history[parcel] = temp[parcel]
    return history


In [ ]:
geoSalesResidFilterSale = geoSalesResid[geoSalesResid.SaleAmount > 100]

saleHistory = getSalesHistory(geoSalesResidFilterSale,False,multi=True)

In [ ]:
# def makeMarker(row,salesDict):
#     parcelnumber = row['ParcelNumber']
#     sales = 'Sales: \n'
#     salesRecord = salesDict[parcelnumber]
#     for saledate in salesRecord:
#         sales += saledate + '  $' + str(salesRecord[saledate])
#         sales += '\n'
    
#     folium.Marker(location=[row['LATITUDE'],row['LONGITUDE']],popup=sales).add_to(m)
#     return

# m = folium.Map(location=[38.0293, -78.4767], zoom_start=13)
# parcels = pd.DataFrame(np.fromiter(saleHistory.keys(),dtype='<U9'))
# filtered = pd.merge(parcels,geoSalesResidFilterSale,left_on=0,right_on="ParcelNumber").drop(columns=[0]).drop_duplicates('ParcelNumber')
# filtered.iloc[:100].apply(makeMarker,axis=1,args=(saleHistory,))
# m
